# Load and Ingest Data
## Load Data

In [3]:
# !wget "https://github.com/ppasupat/WikiTableQuestions/releases/download/v1.0.2/WikiTableQuestions-1.0.2-compact.zip" -O data.zip
# !unzip data.zip


In [4]:
import pandas as pd
from pathlib import Path

data_dir = Path("./WikiTableQuestions/csv/200-csv")
csv_files = sorted([f for f in data_dir.glob("*.csv")])
dfs = []
for csv_file in csv_files:
    print(f"processing file: {csv_file}")
    try:
        df = pd.read_csv(csv_file)
        dfs.append(df)
    except Exception as e:
        print(f"Error parsing {csv_file}: {str(e)}")

processing file: WikiTableQuestions/csv/200-csv/0.csv
processing file: WikiTableQuestions/csv/200-csv/1.csv
processing file: WikiTableQuestions/csv/200-csv/10.csv
processing file: WikiTableQuestions/csv/200-csv/11.csv
processing file: WikiTableQuestions/csv/200-csv/12.csv
processing file: WikiTableQuestions/csv/200-csv/14.csv
processing file: WikiTableQuestions/csv/200-csv/15.csv
Error parsing WikiTableQuestions/csv/200-csv/15.csv: Error tokenizing data. C error: Expected 4 fields in line 16, saw 5

processing file: WikiTableQuestions/csv/200-csv/17.csv
Error parsing WikiTableQuestions/csv/200-csv/17.csv: Error tokenizing data. C error: Expected 6 fields in line 5, saw 7

processing file: WikiTableQuestions/csv/200-csv/18.csv
processing file: WikiTableQuestions/csv/200-csv/20.csv
processing file: WikiTableQuestions/csv/200-csv/22.csv
processing file: WikiTableQuestions/csv/200-csv/24.csv
processing file: WikiTableQuestions/csv/200-csv/25.csv
processing file: WikiTableQuestions/csv/200-

## Extract Table Name and Summary from each Table

In [5]:
tableinfo_dir = "WikiTableQuestions_TableInfo"
!mkdir {tableinfo_dir}


I0000 00:00:1737948939.345619 2580638 fork_posix.cc:77] Other threads are currently calling into gRPC, skipping fork() handlers


mkdir: WikiTableQuestions_TableInfo: File exists


In [6]:
from llama_index.core.program import LLMTextCompletionProgram
from llama_index.core.bridge.pydantic import BaseModel, Field
from llama_index.llms.openai import OpenAI


class TableInfo(BaseModel):
    """Information regarding a structured table."""

    table_name: str = Field(
        ..., description="table name (must be underscores and NO spaces)"
    )
    table_summary: str = Field(
        ..., description="short, concise summary/caption of the table"
    )


prompt_str = """\
Give me a summary of the table with the following JSON format.

- The table name must be unique to the table and describe it while being concise. 
- Do NOT output a generic table name (e.g. table, my_table).

Do NOT make the table name one of the following: {exclude_table_name_list}

Table:
{table_str}

Summary: """

program = LLMTextCompletionProgram.from_defaults(
    output_cls=TableInfo,
    llm=OpenAI(model="gpt-3.5-turbo",api_key="sk-S1h0hPEa3Qa5pCAJHNf6sd9EHxjGSlzd99xOgE4GYRpdBxek",api_base="https://api.chatanywhere.tech/v1"),
    prompt_template_str=prompt_str,
)

In [7]:
import json


def _get_tableinfo_with_index(idx: int) -> str:
    results_gen = Path(tableinfo_dir).glob(f"{idx}_*")
    results_list = list(results_gen)
    if len(results_list) == 0:
        return None
    elif len(results_list) == 1:
        path = results_list[0]
        return TableInfo.parse_file(path)
    else:
        raise ValueError(
            f"More than one file matching index: {list(results_gen)}"
        )


table_names = set()
table_infos = []
for idx, df in enumerate(dfs):
    table_info = _get_tableinfo_with_index(idx)
    if table_info:
        table_infos.append(table_info)
    else:
        while True:
            df_str = df.head(10).to_csv()
            table_info = program(
                table_str=df_str,
                exclude_table_name_list=str(list(table_names)),
            )
            table_name = table_info.table_name
            print(f"Processed table: {table_name}")
            if table_name not in table_names:
                table_names.add(table_name)
                break
            else:
                # try again
                print(f"Table name {table_name} already exists, trying again.")
                pass

        out_file = f"{tableinfo_dir}/{idx}_{table_name}.json"
        json.dump(table_info.dict(), open(out_file, "w"))
    table_infos.append(table_info)

/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)


## Put Data in SQL Database

In [8]:
# put data into sqlite db
from sqlalchemy import (
    create_engine,
    MetaData,
    Table,
    Column,
    String,
    Integer,
)
import re


# Function to create a sanitized column name
def sanitize_column_name(col_name):
    # Remove special characters and replace spaces with underscores
    return re.sub(r"\W+", "_", col_name)


# Function to create a table from a DataFrame using SQLAlchemy
def create_table_from_dataframe(
    df: pd.DataFrame, table_name: str, engine, metadata_obj
):
    # Sanitize column names
    sanitized_columns = {col: sanitize_column_name(col) for col in df.columns}
    df = df.rename(columns=sanitized_columns)

    # Dynamically create columns based on DataFrame columns and data types
    columns = [
        Column(col, String if dtype == "object" else Integer)
        for col, dtype in zip(df.columns, df.dtypes)
    ]

    # Create a table with the defined columns
    table = Table(table_name, metadata_obj, *columns)

    # Create the table in the database
    metadata_obj.create_all(engine)

    # Insert data from DataFrame into the table
    with engine.connect() as conn:
        for _, row in df.iterrows():
            insert_stmt = table.insert().values(**row.to_dict())
            conn.execute(insert_stmt)
        conn.commit()


engine = create_engine("sqlite:///:memory:")
metadata_obj = MetaData()
for idx, df in enumerate(dfs):
    tableinfo = _get_tableinfo_with_index(idx)
    print(f"Creating table: {tableinfo.table_name}")
    create_table_from_dataframe(df, tableinfo.table_name, engine, metadata_obj)

/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)
/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)
/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is depre

Creating table: progression_of_renaissance_albums
Creating table: filmography_of_diane_and_others
Creating table: annual_deaths_and_accidents_statistics
Creating table: academy_and_bafta_awards_1972
Creating table: theatrical_awards_nominations_and_wins
Creating table: bad_boy_artists_album_release_years
Creating table: south_dakota_radio_stations
Creating table: missing_persons_cases_1982
Creating table: mick_jagger_single_chart_performance
Creating table: kodachrome_film_types_and_dates
Creating table: bbc_radio_services_costs_2012_2013
Creating table: french_aerodromes_and_airports_usage


/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)
/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)


Creating table: voter_registration_statistics_by_party
Creating table: norwegian_and_england_club_performance_statistics
Creating table: triple_crown_winners_history
Creating table: grammy_awards_performance_by_artist
Creating table: boxing_fight_results_history
Creating table: historical_college_football_records
Creating table: yamato_prefecture_population_density
Creating table: voter_registration_statistics_by_party_type
Creating table: best_actress_awards_nominations_and_wins
Creating table: uk_ministers_and_titles_history
Creating table: municipalities_and_seat_assignments
Creating table: euro_2020_group_stage_team_performance
Creating table: binary_encoding_implied_probabilities
Creating table: monthly_climate_statistics
Creating table: italian_government_terms_of_office
Creating table: new_mexico_government_officials
Creating table: monthly_climate_statistics_overview
Creating table: rainfall_events_timeline
Creating table: monthly_temperature_and_precipitation_statistics
Creati

/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)
/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is deprecated; load the data from file, then if your data is JSON use `model_validate_json`, otherwise `model_validate` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  return TableInfo.parse_file(path)
/var/folders/7m/hvn7fgns7mg_16dpc1v14wmc0000gn/T/ipykernel_96012/2332672304.py:11: PydanticDeprecatedSince20: The `parse_file` method is depre

In [9]:
# setup Arize Phoenix for logging/observability
import phoenix as px
import llama_index.core

px.launch_app()
llama_index.core.set_global_handler("arize_phoenix")

Existing running Phoenix instance detected! Shutting it down and starting a new instance...


🌍 To view the Phoenix app in your browser, visit http://localhost:6006/
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix


# Advanced Capability 1: Text-to-SQL with Query-Time Table Retrieval.

## Define Modules

#### Object index, retriever, SQLDatabase

In [10]:
from llama_index.core.objects import (
    SQLTableNodeMapping,
    ObjectIndex,
    SQLTableSchema,
)
from llama_index.core import SQLDatabase, VectorStoreIndex

sql_database = SQLDatabase(engine)

table_node_mapping = SQLTableNodeMapping(sql_database)
table_schema_objs = [
    SQLTableSchema(table_name=t.table_name, context_str=t.table_summary)
    for t in table_infos
]  # add a SQLTableSchema for each table

obj_index = ObjectIndex.from_objects(
    table_schema_objs,
    table_node_mapping,
    VectorStoreIndex,
)
obj_retriever = obj_index.as_retriever(similarity_top_k=3)

#### SQLRetriever + Table Parser

In [11]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_str(table_schema_objs: List[SQLTableSchema]):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_str)

#### Text-to-SQL Prompt + Output Parser

In [12]:
from llama_index.core.prompts.default_prompts import DEFAULT_TEXT_TO_SQL_PROMPT
from llama_index.core import PromptTemplate
from llama_index.core.query_pipeline import FnComponent
from llama_index.core.llms import ChatResponse


def parse_response_to_sql(response: ChatResponse) -> str:
    """Parse response to SQL."""
    response = response.message.content
    sql_query_start = response.find("SQLQuery:")
    if sql_query_start != -1:
        response = response[sql_query_start:]
        # TODO: move to removeprefix after Python 3.9+
        if response.startswith("SQLQuery:"):
            response = response[len("SQLQuery:") :]
    sql_result_start = response.find("SQLResult:")
    if sql_result_start != -1:
        response = response[:sql_result_start]
    return response.strip().strip("```").strip()


sql_parser_component = FnComponent(fn=parse_response_to_sql)

text2sql_prompt = DEFAULT_TEXT_TO_SQL_PROMPT.partial_format(
    dialect=engine.dialect.name
)
print(text2sql_prompt.template)

Given an input question, first create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer. You can order the results by a relevant column to return the most interesting examples in the database.

Never query for all the columns from a specific table, only ask for a few relevant columns given the question.

Pay attention to use only the column names that you can see in the schema description. Be careful to not query for columns that do not exist. Pay attention to which column is in which table. Also, qualify column names with the table name when needed. You are required to use the following format, each taking one line:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result of the SQLQuery
Answer: Final answer here

Only use tables listed below.
{schema}

Question: {query_str}
SQLQuery: 


#### Response Synthesis Prompt

In [13]:
response_synthesis_prompt_str = (
    "Given an input question, synthesize a response from the query results.\n"
    "Query: {query_str}\n"
    "SQL: {sql_query}\n"
    "SQL Response: {context_str}\n"
    "Response: "
)
response_synthesis_prompt = PromptTemplate(
    response_synthesis_prompt_str,
)

In [14]:
llm = OpenAI(model="gpt-3.5-turbo",api_key="sk-S1h0hPEa3Qa5pCAJHNf6sd9EHxjGSlzd99xOgE4GYRpdBxek",api_base="https://api.chatanywhere.tech/v1")

## Define Query Pipeline

In [15]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [16]:
qp.add_chain(["input", "table_retriever", "table_output_parser"])
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

## Visualize Query Pipeline

In [17]:
from pyvis.network import Network

net = Network(notebook=True, cdn_resources="in_line", directed=True)
net.from_nx(qp.dag)

In [18]:
# Save the network as "text2sql_dag.html"
net.write_html("text2sql_dag.html")

## Run Some Queries!

In [20]:
response = qp.run(
    query="What was the year that The Notorious B.I.G was signed to Bad Boy?"
)
print(str(response))

> Running module input with input: 
query: What was the year that The Notorious B.I.G was signed to Bad Boy?

> Running module table_retriever with input: 
input: What was the year that The Notorious B.I.G was signed to Bad Boy?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='bad_boy_artists_album_release_years', context_str='List of artists signed to Bad Boy Records with their signing year and number of albums released.'), SQLTableSchema(table_...

> Running module text2sql_prompt with input: 
query_str: What was the year that The Notorious B.I.G was signed to Bad Boy?
schema: Table 'bad_boy_artists_album_release_years' has columns: Act (VARCHAR), Year_signed (INTEGER), _Albums_released_under_Bad_Boy (VARCHAR), . The table description is: List of artists signed to Bad Boy R...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results

In [21]:
response = qp.run(query="Who won best director in the 1972 academy awards")
print(str(response))

> Running module input with input: 
query: Who won best director in the 1972 academy awards

> Running module table_retriever with input: 
input: Who won best director in the 1972 academy awards

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='academy_and_bafta_awards_1972', context_str='Awards and nominations for the 1972 Academy and BAFTA ceremonies.'), SQLTableSchema(table_name='best_actress_awards_nominations...

> Running module text2sql_prompt with input: 
query_str: Who won best director in the 1972 academy awards
schema: Table 'academy_and_bafta_awards_1972' has columns: Award (VARCHAR), Category (VARCHAR), Nominee (VARCHAR), Result (VARCHAR), . The table description is: Awards and nominations for the 1972 Academy and...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order 

In [22]:
response = qp.run(query="What was the term of Pasquale Preziosa?")
print(str(response))

> Running module input with input: 
query: What was the term of Pasquale Preziosa?

> Running module table_retriever with input: 
input: What was the term of Pasquale Preziosa?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='italian_government_terms_of_office', context_str='List of individuals and their terms in Italian government positions from 1926 to 1951.'), SQLTableSchema(table_name='uk_mi...

> Running module text2sql_prompt with input: 
query_str: What was the term of Pasquale Preziosa?
schema: Table 'italian_government_terms_of_office' has columns: Name (VARCHAR), Term_start (VARCHAR), Term_end (VARCHAR), . The table description is: List of individuals and their terms in Italian government ...

> Running module text2sql_llm with input: 
messages: Given an input question, first create a syntactically correct sqlite query to run, then look at the results of the query and return the answer. You can order the results by a relevant c

In [25]:
response = qp.run(query="Who was the Italian president named Pasquale Preziosa?When did Pasquale Preziosa begin his presidency?How long did Pasquale Preziosa’s presidency last?")
print(str(response))

> Running module input with input: 
query: Who was the Italian president named Pasquale Preziosa?When did Pasquale Preziosa begin his presidency?How long did Pasquale Preziosa’s presidency last?

> Running module table_retriever with input: 
input: Who was the Italian president named Pasquale Preziosa?When did Pasquale Preziosa begin his presidency?How long did Pasquale Preziosa’s presidency last?

> Running module table_output_parser with input: 
table_schema_objs: [SQLTableSchema(table_name='italian_government_terms_of_office', context_str='List of individuals and their terms in Italian government positions from 1926 to 1951.'), SQLTableSchema(table_name='uk_mi...

> Running module text2sql_prompt with input: 
query_str: Who was the Italian president named Pasquale Preziosa?When did Pasquale Preziosa begin his presidency?How long did Pasquale Preziosa’s presidency last?
schema: Table 'italian_government_terms_of_office' has columns: Name (VARCHAR), Term_start (VARCHAR), Term_end (VAR

# Advanced Capability 2: Text-to-SQL with Query-Time Row Retrieval (along with Table Retrieval)

## Index Each Table

In [26]:
from llama_index.core import VectorStoreIndex, load_index_from_storage
from sqlalchemy import text
from llama_index.core.schema import TextNode
from llama_index.core import StorageContext
import os
from pathlib import Path
from typing import Dict


def index_all_tables(
    sql_database: SQLDatabase, table_index_dir: str = "table_index_dir"
) -> Dict[str, VectorStoreIndex]:
    """Index all tables."""
    if not Path(table_index_dir).exists():
        os.makedirs(table_index_dir)

    vector_index_dict = {}
    engine = sql_database.engine
    for table_name in sql_database.get_usable_table_names():
        print(f"Indexing rows in table: {table_name}")
        if not os.path.exists(f"{table_index_dir}/{table_name}"):
            # get all rows from table
            with engine.connect() as conn:
                cursor = conn.execute(text(f'SELECT * FROM "{table_name}"'))
                result = cursor.fetchall()
                row_tups = []
                for row in result:
                    row_tups.append(tuple(row))

            # index each row, put into vector store index
            nodes = [TextNode(text=str(t)) for t in row_tups]

            # put into vector store index (use OpenAIEmbeddings by default)
            index = VectorStoreIndex(nodes)

            # save index
            index.set_index_id("vector_index")
            index.storage_context.persist(f"{table_index_dir}/{table_name}")
        else:
            # rebuild storage context
            storage_context = StorageContext.from_defaults(
                persist_dir=f"{table_index_dir}/{table_name}"
            )
            # load index
            index = load_index_from_storage(
                storage_context, index_id="vector_index"
            )
        vector_index_dict[table_name] = index

    return vector_index_dict


vector_index_dict = index_all_tables(sql_database)

Indexing rows in table: academy_and_bafta_awards_1972
Indexing rows in table: annual_deaths_and_accidents_statistics
Indexing rows in table: bad_boy_artists_album_release_years
Indexing rows in table: bbc_radio_services_costs_2012_2013
Indexing rows in table: best_actress_awards_nominations_and_wins
Indexing rows in table: binary_encoding_implied_probabilities
Indexing rows in table: boxing_fight_results_history
Indexing rows in table: cancer_related_genetic_factors_and_functions
Indexing rows in table: euro_2020_group_stage_team_performance
Indexing rows in table: filmography_of_diane_and_others
Indexing rows in table: french_aerodromes_and_airports_usage
Indexing rows in table: grammy_awards_performance_by_artist
Indexing rows in table: historical_college_football_records
Indexing rows in table: italian_government_terms_of_office
Indexing rows in table: kodachrome_film_types_and_dates
Indexing rows in table: mick_jagger_single_chart_performance
Indexing rows in table: missing_persons

## Define Expanded Table Parser Component

In [27]:
from llama_index.core.retrievers import SQLRetriever
from typing import List
from llama_index.core.query_pipeline import FnComponent

sql_retriever = SQLRetriever(sql_database)


def get_table_context_and_rows_str(
    query_str: str, table_schema_objs: List[SQLTableSchema]
):
    """Get table context string."""
    context_strs = []
    for table_schema_obj in table_schema_objs:
        # first append table info + additional context
        table_info = sql_database.get_single_table_info(
            table_schema_obj.table_name
        )
        if table_schema_obj.context_str:
            table_opt_context = " The table description is: "
            table_opt_context += table_schema_obj.context_str
            table_info += table_opt_context

        # also lookup vector index to return relevant table rows
        vector_retriever = vector_index_dict[
            table_schema_obj.table_name
        ].as_retriever(similarity_top_k=2)
        relevant_nodes = vector_retriever.retrieve(query_str)
        if len(relevant_nodes) > 0:
            table_row_context = "\nHere are some relevant example rows (values in the same order as columns above)\n"
            for node in relevant_nodes:
                table_row_context += str(node.get_content()) + "\n"
            table_info += table_row_context

        context_strs.append(table_info)
    return "\n\n".join(context_strs)


table_parser_component = FnComponent(fn=get_table_context_and_rows_str)

## Define Expanded Query Pipeline

In [28]:
from llama_index.core.query_pipeline import (
    QueryPipeline as QP,
    Link,
    InputComponent,
    CustomQueryComponent,
)

qp = QP(
    modules={
        "input": InputComponent(),
        "table_retriever": obj_retriever,
        "table_output_parser": table_parser_component,
        "text2sql_prompt": text2sql_prompt,
        "text2sql_llm": llm,
        "sql_output_parser": sql_parser_component,
        "sql_retriever": sql_retriever,
        "response_synthesis_prompt": response_synthesis_prompt,
        "response_synthesis_llm": llm,
    },
    verbose=True,
)

In [29]:
qp.add_link("input", "table_retriever")
qp.add_link("input", "table_output_parser", dest_key="query_str")
qp.add_link(
    "table_retriever", "table_output_parser", dest_key="table_schema_objs"
)
qp.add_link("input", "text2sql_prompt", dest_key="query_str")
qp.add_link("table_output_parser", "text2sql_prompt", dest_key="schema")
qp.add_chain(
    ["text2sql_prompt", "text2sql_llm", "sql_output_parser", "sql_retriever"]
)
qp.add_link(
    "sql_output_parser", "response_synthesis_prompt", dest_key="sql_query"
)
qp.add_link(
    "sql_retriever", "response_synthesis_prompt", dest_key="context_str"
)
qp.add_link("input", "response_synthesis_prompt", dest_key="query_str")
qp.add_link("response_synthesis_prompt", "response_synthesis_llm")

In [31]:
# from pyvis.network import Network

# net = Network(notebook=True, cdn_resources="in_line", directed=True)
# net.from_nx(qp.dag)
# net.show("text2sql_dag.html")

## Run Some Queries

In [32]:
response = qp.run(
    query="What was the year that The Notorious BIG was signed to Bad Boy?"
)
print(str(response))

> Running module input with input: 
query: What was the year that The Notorious BIG was signed to Bad Boy?

> Running module table_retriever with input: 
input: What was the year that The Notorious BIG was signed to Bad Boy?

> Running module table_output_parser with input: 
query_str: What was the year that The Notorious BIG was signed to Bad Boy?
table_schema_objs: [SQLTableSchema(table_name='bad_boy_artists_album_release_years', context_str='List of artists signed to Bad Boy Records with their signing year and number of albums released.'), SQLTableSchema(table_...

> Running module text2sql_prompt with input: 
query_str: What was the year that The Notorious BIG was signed to Bad Boy?
schema: Table 'bad_boy_artists_album_release_years' has columns: Act (VARCHAR), Year_signed (INTEGER), _Albums_released_under_Bad_Boy (VARCHAR), . The table description is: List of artists signed to Bad Boy R...

> Running module text2sql_llm with input: 
messages: Given an input question, first create 